In [ ]:
import pandas as pd
import numpy as np
import cvxpy
import os.path
import data
from pandas.io.data import DataReader

auth = "d85AxYgsEaWPyhx_k2ZL"

def yahoo_prices(symbols, start_date, verbose = True):
    ticker_df_list = []
    for index, row in symbols.iterrows(): 
        try:
            data = DataReader(row.Ticker, 'yahoo', start_date)
            data['Ref'] = row.Ticker 
            data = data.loc[:, ['Ref', row.PriceColumn]]
            data.rename(columns={row.PriceColumn: 'Price'}, inplace=True)
            ticker_df_list.append(data)
            if verbose:
                print("Obtained data for ticker %s" % row.Ticker)
        except Exception as e:
            if verbose:
                print("No data for ticker %s\n%s" % (row.Ticker, str(e)))    
    df = pd.concat(ticker_df_list)   
    cell= df[['Ref','Price']] 
    return cell.pivot(columns='Ref')


def mean_variance(returns, samples=100):
    # Setup the optimization problem
    p = returns.mean().as_matrix()
    w = cvxpy.Variable(len(p))
    gamma = cvxpy.Parameter(sign='positive')
    
    
    ret = p * w
    
    covs = rets.cov().as_matrix()
    sigma = cvxpy.quad_form(w, covs)
    
    prob = cvxpy.Problem(cvxpy.Maximize(ret - gamma*sigma),
                         [cvxpy.sum_entries(w)==1, w>=0])

    # Compute trade-off curve.
    SAMPLES = 100
    solutions = []
    gamma_vals = np.logspace(-2, 3, num=SAMPLES)
    for i in range(SAMPLES):
        gamma.value = gamma_vals[i]
        prob.solve()
        solutions.append({'mean':ret.value, 'sd':cvxpy.sqrt(sigma).value, 'wgts':w.value})
        
    return(solutions)

def plot_portfolios(rets, sols):
    fig = plt.figure()
    
    for i in sols:
        plt.plot(i['sd'], i['mean'], 'bs')
        
    p = rets.mean().as_matrix()
    covs = rets.cov().as_matrix()
    for i in range(len(p)):
        plt.plot(cvxpy.sqrt(covs[i,i]).value, p[i], 'ro')
        
    plt.xlabel('Standard deviation')
    plt.ylabel('Return')
    plt.show()
    
force_download = True

symbols = pd.read_csv('AssetUniverse.csv')
symbols = symbols.loc[:,['Ticker', 'Quandl', 'PriceColumn']]
           
if(force_download or not os.path.exists('fixings.pkl')):
    fixings = data.yahoo_prices(symbols, start_date='2015-1-1')
    fixings.to_pickle('fixings.pkl')              
else:
    fixings = pd.read_pickle('fiximgs.pkl')

# Do we wante to resample to just weekly/monthly fixings?
resample = fixings.resample('W-MON')

rets = resample.pct_change(fill_method='pad')

sol = mean_variance(rets)
#%%
plot_portfolios(rets, sol)


/home/tom/anaconda3/lib/python3.5/site-packages/pandas/io/data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)
